In [593]:
import pandas as pd
import numpy as np
import sys
import datetime
import re

In [228]:
# For debugging
def clean_output_midpoint(output_location):
    # 1: Import the data
    orig = import_output(output_location)
    # 2: Remove all unnecessary columns
    updated = remove_bad_columns(orig)
    # 3: Clean data so it's easier to analyze
    updated = clean_data(updated)
    # 4: Remove invalid rows
    updated = remove_invalid_rows(updated)
    # 5: Obtain information on the reported time
    updated = edit_reported_time(updated)
    
    return updated

# For debugging
def clean_output_secondhalf(df):
    updated = edit_occurred_time(df)
    
    return updated

# Main function
def clean_output(output_location):
    # 1: Import the data
    orig = import_output(output_location)
    # 2: Remove all unnecessary columns
    updated = remove_bad_columns(orig)
    # 3: Clean data so it's easier to analyze
    updated = clean_data(updated)
    # 4: Remove invalid rows
    updated = remove_invalid_rows(updated)
    # 5: Obtain information on the reported time
    updated = edit_reported_time(updated)
    # 6: Obtain information on the occurred time
    updated = edit_occurred_time(updated)
    
    return updated

In [574]:
def import_output(output_location):
    df = pd.read_csv(output_location)
    return df

def remove_bad_columns(df):
    if 'Unnamed: 0' in df.columns:
        df = df.drop(['Unnamed: 0'], axis=1)
    return df

def clean_data(df):
    # We will make all of these values lowercase to make sure there are no capitalization errors.
    columns = ['Incident', 'Location', 'Reported', 'Occurred', 'Disposition']
    for column in columns:
        df[column] = df[column].str.lower().str.strip()
    return df

def remove_invalid_rows(df):
    # Removes all invalid rows - rows that were generated as tests, in error, or that don't have any info.
    df = df[df['Incident'].str.contains('void') == False]
    df = df[df['Incident'].str.contains('no reports this date') == False]
    df = df[df['Disposition'].str.contains('no incident reports this date') == False]
    return df

def clean_time_column(df, column):
    # Helper function to clean some systematic edge cases in time columns. 
    df[column] = df[column].str.replace('!', '1', regex=False)        # Typo; logical answer would be a 1.
    df[column] = df[column].str.replace(' at ', ' ', regex=False)     # Filler words
    df[column] = df[column].str.replace('//', '/', regex=False)       # Sometimes there are additional /'s
    df[column] = df[column].str.replace('a m', 'am', regex=False)     # Sometimes there are accidental spaces
    df[column] = df[column].str.replace('p m', 'pm', regex=False)     # Sometimes there are accidental spaces
    df[column] = df[column].str.replace('am am', 'am', regex=False)   # Sometimes am is repeated
    df[column] = df[column].str.replace('pm pm', 'pm', regex=False)   # Sometimes pm is repeated
    df[column] = df[column].str.replace('a.m.', 'am', regex=False)    # Sometimes there are periods
    df[column] = df[column].str.replace('p.m.', 'pm', regex=False)    # Sometimes there are periods
    df[column] = df[column].str.replace('`', '', regex=False)         # Some weird typo
    df[column] = df[column].str.replace('0112', '12', regex=False)    # A lot of typos on 2012 for some reason
    df[column] = df[column].str.replace('0121', '12', regex=False)    # A lot of typos on 2012 for some reason
    df[column] = df[column].str.replace('(', '', regex=False)         # Makes it easier to analyze
    df[column] = df[column].str.replace(')', '', regex=False)         # Makes it easier to analyze
    
    return df

def edit_reported_time(df):
    # Creates a new column called 'Reported_datetime' which is the reported column, in datetime format.
    df['Reported_datetime'] = pd.to_datetime('')
    
    # Cleaning systematic problems
    df = clean_time_column(df, 'Reported')
    
    # Some one-off edge cases
    # Note to self: Fix the 41658.01736 one. That one isn't fixed correctly.
    replacements = {'4/16/2012 3.38 pm': '4/16/12 3:38 pm', '7/17/12 9:33m': '7/17/12 9:33 pm',
                    '11/5/13 9:54:pm': '11/5/13 9:54 pm', '41659.01736': '1/19/14 1:23 am',
                    '3/27/14 10;11 am': '3/27/14 10:11 am', '8/31/16 5;03 pm': '8/31/16 5:03 pm',
                    '7/19/1810:39 am': '7/19/18 10:39 am', '. 5/15/20 3:04 pm': '5/15/20 3:04 pm',
                    '5/17/21/ 1:04 pm': '5/17/21 1:04 pm'}
    for key, value in replacements.items():
        df['Reported'] = df['Reported'].str.replace(key, value, regex=False)
    
    # Keep as YY, not YYYY
    for year in range(2000, 2099):
        year = str(year)
        df['Reported'] = df['Reported'].str.replace(year, year[-2:], regex=False)
    
    # Fill column which houses reported time, now in datetime format.
    for index, row in df.iterrows():
        reported = row['Reported']
        try:
            if 'am' in reported or 'pm' in reported:
                reported_datetime = datetime.datetime.strptime(reported, "%m/%d/%y %I:%M %p")
            else:
                reported_datetime = datetime.datetime.strptime(reported, "%m/%d/%y %H:%M")
            df.loc[index, 'Reported_datetime'] = reported_datetime
        except:
            print("edit_reported_time: Date error:", reported)
            df.loc[index, 'Reported_datetime'] = pd.to_datetime('')   # NaT
        
        
    
    return df

def edit_occurred_time(df):
    # Occurred can be a range of values or one single value. As such, edit_occurred_time creates three new columns:
    # 1) Occurred_range_or_exact: a column which is either 'range' or 'exact', denoting whether the Occurred column
    # has an exact timestamp or a range.
    # 2) Occurred_lower_range: the lower range for the time of the incident, in datetime.
    # 3) Occurred_upper_range: the upper range for the time of the incident, in datetime.
    # When occurred_range_or_exact == 'range', occurred_lower_range < occurred_upper_range. 
    # When occurred_range_or_exact == 'exact', occurred_lower_range == occurred_upper_range.
    
    df['Occurred_range_or_exact'] = ''
    df['Occurred_lower_range'] = pd.to_datetime('')
    df['Occurred_upper_range'] = pd.to_datetime('')
    
    ### Let's figure out whether it is a range or not.
    # First, define the values that make the cell considered a range, if the cell contains that value.
    range_markers = ['to', '-', 'through', 'thru', 'time unknown', 'times unknown', 'unknown time', 
                     'unspecified time', 'time not specified', 'various times', 'and']
    for range_marker in range_markers:
        df.loc[df['Occurred'].str.contains(range_marker) == True, 'Occurred_range_or_exact'] = 'range'
    
    # Sometimes the string just ends in unknown, signifying that the time is unknown (but the date is known)
    df.loc[df['Occurred'].str[-8:] == ' unknown', 'Occurred_range_or_exact'] = 'range'
    
    # Sometimes the best way to check if a value is a range is if there are two time values (aka two :'s)
    df.loc[df['Occurred'].str.count(':') >= 2, 'Occurred_range_or_exact'] = 'range'
    
    # All other cases...
    df.loc[df['Occurred_range_or_exact'] == '', 'Occurred_range_or_exact'] = 'exact'
    
    ### ...Edge case if the occurred time is unknown.
    df.loc[df['Occurred'] == 'unknown', 'Occurred_range_or_exact'] = 'unknown'
    # Define the values that mark the occurrence time as being unknown
    unknown_markers = ['unknown date and time', 'unknown dates and time', 'various dates and times', 
                       'unknown date/time', 'and various other dates', 'unspecified day and time', 
                       'unknown dates/times', 'date and time unknown', 'spring', 'summer', 'winter', 'fall']
    for unknown_marker in unknown_markers:
        df.loc[df['Occurred'].str.contains(unknown_marker) == True, 'Occurred_range_or_exact'] = 'unknown'
    # Define lower and upper range as NaT, to allow these columns to stay as datetime types.
    df.loc[df['Occurred_range_or_exact'] == 'unknown', 'Occurred_lower_range'] = pd.to_datetime('')
    df.loc[df['Occurred_range_or_exact'] == 'unknown', 'Occurred_upper_range'] = pd.to_datetime('')
    
    ### Let's convert all times that are exact to datetime.
    # First, systematic cleaning.
    df = clean_time_column(df, 'Occurred')
    
    # Clean up the edge cases
    replacements = {'41658.9375': '5/17/21 1:04 pm', '11212 10:52 am': '11/2/12 10:52 am',
                    '1/1/0713 1:29 pm': '1/17/2013 1:29 pm', '7/12/13 11:14:pm': '7/12/13 11:14 pm',
                    '4/20/14 6:20 pm:00': '4/20/14 6:20 pm', '10/25/15 2:53 am)': '10/25/15 2:53 am',
                    '1/17/16 1130 am': '1/17/16 11:30 am', '4/25 16 1:45 pm': '4/25/16 1:45 pm',
                    '9/13/16 14:55 pm': '9/13/16 2:55 pm', '4/13/17 0950': '4/13/17 09:50',
                    '3/17/18b 2:47 pm': '3/17/18 2:47 pm', '5532 s. kenwood 1:28 am': '8/31/19 1:28 am',
                    '7/7/202:45 pm': '7/7/20 2:45 pm', '7/29/21pm': '7/29/21 4:48 pm',
                    '8/112/13 7:18 pm': '8/12/13 7:18 pm'}
    for key, value in replacements.items():
        df['Occurred'] = df['Occurred'].str.replace(key, value, regex=False)
    
    # Keep as YY, not YYYY
    for year in range(2000, 2099):
        year = str(year)
        df['Occurred'] = df['Occurred'].str.replace(year, year[-2:], regex=False)
    
    for index, row in df.iterrows():
        occurred = row['Occurred']
        if row['Occurred_range_or_exact'] == 'exact':
            try:
                if 'am' in occurred or 'pm' in occurred:
                    occurred_datetime = datetime.datetime.strptime(occurred, "%m/%d/%y %I:%M %p")
                else:
                    occurred_datetime = datetime.datetime.strptime(occurred, "%m/%d/%y %H:%M")
                df.loc[index, 'Occurred_lower_range'] = occurred_datetime
                df.loc[index, 'Occurred_upper_range'] = occurred_datetime
            except:
                print(index, "Error with date:", occurred)
                df.loc[index, 'Occurred_lower_range'] = pd.to_datetime('') # NaT
                df.loc[index, 'Occurred_upper_range'] = pd.to_datetime('') # NaT    
    
    ### Let's convert all the times that are ranges to datetime.
    
    # Edge cases
    replacements = {'7/1910 to 7/21/10 8:00 pm to 2:00 pm': '7/19/10 to 7/21/10 8:00 pm to 2:00 pm',
                    '7/31/10 to 8/1/10 9:00 pm to 9: 15 am': '7/31/10 to 8/1/10 9:00 pm to 9:15 am',
                    '10/23/10 to 10/24/10 8:00 am to 9:30:pm': '10/23/10 to 10/24/10 8:00 am to 9:30 pm',
                    '4/13/11 to 4/14/11 6: 20 pm to 5:30 am': '4/13/11 to 4/14/11 6:20 pm to 5:30 am',
                    '10/2811 to 10/29/11 12:40 pm to 10:00 am': '10/28/11 to 10/29/11 12:40 pm to 10:00 am',
                    '1/11/12 to 1/12/12/ 6:00pm to 11:15am': '1/11/12 to 1/12/12 6:00pm to 11:15am',
                    '1/12/12 to 1/12/12/ 7:30pm to 11:00pm': '1/12/12 to 1/12/12 7:30pm to 11:00pm',
                    '2/10/11 t0 2/21/11 1:30 pm to 7:00 am': '2/10/11 to 2/21/11 1:30 pm to 7:00 am',
                    '1/17:12 1:00pm to 10:00pm': '1/17/12 1:00pm to 10:00pm',
                    '6/14/112 8:00 am to 5:30 pm': '6/14/12 8:00 am to 5:30 pm',
                    '9/20/16 t0 9/21/16 12:00 pm to 3:00 pm': '9/20/16 to 9/21/16 12:00 pm to 3:00 pm',
                    '5/31/12 10:445 am to 6:00 pm': '5/31/12 10:45 am to 6:00 pm',
                    '6/23/11 11:15 am to 11:45am': '6/23/11 11:15 am to 11:45 am',
                    '11/23/10 10 :30 am to 2:30 pm': '11/23/10 10:30 am to 2:30 pm',
                    '10/25/13 5:445 pm to 7:25 pm': '10/25/13 5:45 pm to 7:25 pm'}
    for key, value in replacements.items():
        df['Occurred'] = df['Occurred'].str.replace(key, value, regex=False)
        
    for index, row in df.iterrows():
        occurred = row['Occurred']
        occurred_original = occurred
        
        # get rid of some superfluous words at the end
        superfluous_words = ['various', 'time not specified', 'unspecified time', 'unknown time',
                             'time unknown', 'times unknown', 'periodic', 'unknown']
        for word in superfluous_words:
            if word in occurred:
                i = occurred.find(word)
                occurred = occurred[:i].strip()
        
        # putting spaces around all 'to's, as long as it isn't 'october'
        if 'to' in occurred:
            indeces = [i for i in range(len(occurred)) if occurred.startswith('to', i)]
            indeces.reverse()   # need to reverse as we're mutating the string while also working w/ its indeces
            for i in indeces:
                if occurred[i-1] != ' ':
                    occurred = occurred[:i] + ' ' + occurred[i:]
                if occurred[i+2] != ' ':
                    occurred = occurred[:i+2] + ' ' + occurred[i+2:]
                occurred = occurred.replace('oc to ber', 'october') # fix october
                
        # Trimming some of the times; we want hh:mm, not hh:mm:ss
        matches = [m.span() for m in re.finditer('\d:\d\d:00 ', occurred)]
        matches.reverse()   # Reverse because we're indexing and editing the same string.
        for match in matches:
            i = match[0]
            occurred = occurred[:i+4] + occurred[i+7:]
        
        # useful variables
        occurred_split = occurred.split(' ')
        to_count = occurred.count('to') - occurred.count('october')
        emdash_count = occurred.count('-')
        
        if row['Occurred_range_or_exact'] == 'range':
            try:
                if to_count == 2:
                    if len(occurred_split) == 8:
                        lowerbound = occurred_split[0] + ' ' + occurred_split[3] + ' ' + occurred_split[4]
                        upperbound = occurred_split[2] + ' ' + occurred_split[6] + ' ' + occurred_split[7]
                    elif len(occurred_split) == 6:
                        lowerbound = occurred_split[0] + ' ' + occurred_split[3][:-2] + ' ' + occurred_split[3][-2:]
                        upperbound = occurred_split[2] + ' ' + occurred_split[5][:-2] + ' ' + occurred_split[5][-2:]
                    else:
                        raise Exception('undefined occurred_split length, count(to) == 2')
                elif to_count == 1 or emdash_count == 1:
                    if len(occurred_split) == 7:
                        lowerbound = occurred_split[0] + ' ' + occurred_split[1] + ' ' + occurred_split[2]
                        upperbound = occurred_split[4] + ' ' + occurred_split[5] + ' ' + occurred_split[6]
                    elif len(occurred_split) == 6:
                        lowerbound = occurred_split[0] + ' ' + occurred_split[1] + ' ' + occurred_split[2]
                        upperbound = occurred_split[0] + ' ' + occurred_split[4] + ' ' + occurred_split[5]
                    elif len(occurred_split) == 4:
                        lowerbound = occurred_split[0] + ' ' + occurred_split[1][:-2] + ' ' + occurred_split[1][-2:]
                        upperbound = occurred_split[0] + ' ' + occurred_split[3][:-2] + ' ' + occurred_split[3][-2:]
                    elif len(occurred_split) == 3:
                        lowerbound = occurred_split[0] + ' 12:00 am'
                        upperbound = occurred_split[2] + ' 11:59 pm'
                    else:
                        raise Exception('undefined occurred_split length, count(to) == 1')
                elif to_count == 0:
                    if len(occurred_split) == 1:
                        lowerbound = occurred_split[0] + ' 12:00 am'
                        upperbound = occurred_split[0] + ' 11:59 pm'
                else:
                    raise Exception('undefined occurred_split length')
                    
                lowerbound_datetime = datetime.datetime.strptime(lowerbound, "%m/%d/%y %I:%M %p")
                upperbound_datetime = datetime.datetime.strptime(upperbound, "%m/%d/%y %I:%M %p")
                df.loc[index, 'Occurred_lower_range'] = lowerbound_datetime
                df.loc[index, 'Occurred_upper_range'] = upperbound_datetime
                
            except:
                print(index, "edit_occurred_time: Date error:", to_count, 
                      '(' + occurred + ')', '(' + occurred_original + ')')
                df.loc[index, 'Occurred_lower_range'] = pd.to_datetime('') # NaT
                df.loc[index, 'Occurred_upper_range'] = pd.to_datetime('') # NaT

    return df

In [571]:
# For debugging
temp = clean_output_midpoint('ucpddata-070110-123121.csv')
cleaned = clean_output_secondhalf(temp)
# cleaned.head(100)

edit_reported_time: Date error: 2/18/11 3:26 am
edit_reported_time: Date error: 6/23/15 12:16 pm


KeyboardInterrupt: 

In [575]:
# Main function
output_location = 'ucpddata-070110-123121.csv'
cleaned = clean_output(output_location)
cleaned.to_csv('cleaned-070110-123121.csv')

381 edit_occurred_time: Date error: 1 (9/17/10 0750 to 4:50 pm) (9/17/10 0750 to 4:50 pm)
1462 edit_occurred_time: Date error: 1 (6/30 /11 7:00 am to 10:30 am) (6/30 /11 7:00 am to 10:30 am)
1629 edit_occurred_time: Date error: 1 (8/5/11 2:25 to 2:35 am) (8/5/11 2:25 to 2:35 am)
1730 edit_occurred_time: Date error: 1 (8/15/11 to 9/10/11 7:00 pm) (8/15/11 to 9/10/11 7:00 pm)
1770 edit_occurred_time: Date error: 1 (9/23/11 to 9/24/11 2:25 am) (9/23/11 to 9/24/11 2:25 am)
1931 edit_occurred_time: Date error: 1 (11/7/11 7:05 to 7:30 am) (11/7/11 7:05 to 7:30 am)
2031 edit_occurred_time: Date error: 1 (12/7/11 to 12/12/11 10:30 am 8:30 am) (12/7/11 to 12/12/11 10:30 am 8:30 am)
2161 edit_occurred_time: Date error: 1 (1/27/12 1:30am to 1:40 am) (1/27/12 1:30am to 1:40 am)
2272 edit_occurred_time: Date error: 2 (3/3/112 to 3/4/12 6:30 pm to 8:50 am) (3/3/112 to 3/4/12 6:30 pm to 8:50 am)
2279 edit_occurred_time: Date error: 2 (3/5/12 to 3/6/121 6:00 pm to 7:30 am) (3/5/12 to 3/6/121 6:00 pm t

In [566]:
cleaned.to_csv('cleaned-070110-123121.csv')

In [599]:
1 - len(cleaned[cleaned['Occurred_lower_range'].isnull()]) / len(cleaned)

0.9926087568632972